##### Copyright 2022 The Cirq Developers

In [ ]:
#2025-02-16 17:30:52.504060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow #binary is optimized to use available CPU instructions in performance-critical operations.
#To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the #appropriate compiler flags.
#MCMC Progress:  30%|███       | 3/10 [00:00<00:00, 22.93sample/s]
#Szukanie wzorca dla seeda1: 2024-11-26 20:15:08
#Rozpoczynam wyszukiwanie wzorca [2, 9, 16, 46, 47] przy użyciu MCMC...
#MCMC Progress: 100%|██████████| 10/10 [00:00<00:00, 22.96sample/s]
#MCMC Progress:  30%|███       | 3/10 [00:00<00:00, 24.00sample/s]
#
#Nie znaleziono wzorca w wygenerowanych próbkach.
#
#Szukanie wzorca dla seeda2: 2024-12-03 20:15:08
#Rozpoczynam wyszukiwanie wzorca [2, 9, 16, 46, 47] przy użyciu MCMC...
#MCMC Progress: 100%|██████████| 10/10 [00:00<00:00, 23.45sample/s]
import sys
import os
import random
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import pandas as pd

# Ustawienia liczby rdzeni CPU dla TensorFlow
# tf.config.threading.set_intra_op_parallelism_threads(8)
# tf.config.threading.set_inter_op_parallelism_threads(8)

# Ustawienia seedów
seed1_datetime = datetime.strptime("2024-11-26 20:15:08", "%Y-%m-%d %H:%M:%S") # 20 21 28 32 37 . 01 04
seed2_datetime = datetime.strptime("2024-12-03 20:15:08", "%Y-%m-%d %H:%M:%S") # 07 20 23 24 37 . 04 10
seed1 = int(seed1_datetime.timestamp())
seed2 = int(seed2_datetime.timestamp())

np.random.seed(seed1)
tf.random.set_seed(seed1)

# Definiowanie target pattern
# target_pattern = [02,09,16,46,47]
target_pattern = [2,9,16,46,47]
iRow = 999_999_999_999_999

# Generowanie danych (zamiast wczytywania z pliku)
data_content = [
[5,8,21,37,46],
[5,7,12,19,26],
[7,8,34,36,38],
[5,11,12,27,32],
[10,16,30,41,45],
[10,13,19,40,45],
[29,30,35,41,45],
[15,21,38,39,47],
[12,18,43,44,46],
[4,11,26,32,41],
[11,28,34,37,38],
[10,15,16,36,44],
[6,10,17,18,42],
[13,14,16,47,49],
[5,8,19,33,36],
[10,22,24,33,38],
[8,12,27,31,37],
[6,9,13,14,25],
[9,22,23,34,35],
[12,15,18,24,25],
[6,11,40,46,48],
[3,16,30,34,35],
[9,20,21,33,49],
[8,22,28,40,42],
[15,23,25,46,49],
[10,14,15,24,47],
[18,29,41,44,49],
[5,14,16,34,39],
[26,28,40,49,50],
[6,11,17,43,49],
[6,14,18,19,48],
[7,17,18,19,22],
[3,5,7,12,31],
[25,29,38,41,44],
[1,4,7,21,26],
[1,21,32,36,50],
[2,13,21,24,37],
[13,20,22,27,42],
[17,28,31,35,39],
[11,20,24,25,35],
[5,8,22,32,39],
[6,20,29,40,45],
[6,31,37,41,49],
[9,13,21,28,45],
[4,13,28,37,38],
[2,4,14,26,29],
[12,14,17,27,43],
[4,9,26,32,40],
[3,13,41,46,49],
[19,28,29,34,44],
[10,12,14,30,43],
[4,30,32,41,50],
[5,12,21,43,48],
[1,22,27,37,50],
[22,31,37,38,41],
[8,19,22,32,40],
[17,18,22,24,25],
[11,14,19,23,49],
[14,16,28,44,48],
[6,9,31,33,35],
[4,6,10,13,32],
[2,4,9,20,41],
[5,9,12,23,40],
[6,9,11,29,35],
[1,25,46,47,48],
[5,14,21,23,50],
[6,8,23,33,49],
[15,16,17,19,50],
[7,27,33,47,48],
[7,18,19,33,37],
[4,13,19,25,43],
[14,23,29,33,37],
[7,26,32,34,49],
[5,32,33,38,49],
[4,10,25,30,47],
[6,9,10,17,43],
[3,7,20,32,44],
[3,10,18,31,43],
[2,6,15,18,24],
[6,10,21,34,36],
[1,7,16,21,48],
[2,11,23,29,34],
[7,22,24,25,41],
[9,25,39,41,46],
[7,10,13,26,27],
[18,25,28,34,42],
[3,19,32,43,48],
[1,38,39,42,47],
[17,25,32,35,45],
[1,8,18,25,47],
[9,19,24,28,49],
[9,12,22,37,40],
[2,15,33,40,43],
[1,16,34,36,37],
[8,18,22,33,40],
[4,7,15,25,29],
[4,28,30,36,46],
[12,17,29,39,40],
[16,18,20,36,49],
[19,21,22,28,45],
[2,4,21,30,41],
[2,17,21,22,45],
[7,14,15,20,39],
[6,7,9,13,30],
[8,18,35,38,42],
[10,32,35,38,45],
[10,16,28,30,33],
[1,7,14,22,44],
[12,32,40,44,49],
[4,7,10,39,48],
[9,31,32,33,34],
[1,11,14,22,43],
[9,28,36,42,43],
[6,21,33,37,47],
[7,17,25,26,35],
[6,9,19,37,38],
[1,8,15,18,39],
[6,10,23,33,41],
[11,33,35,42,44],
[3,10,11,13,42],
[3,10,16,35,45],
[6,11,15,18,45],
[3,10,36,43,47],
[5,18,33,38,47],
[15,19,21,34,40],
[17,38,41,43,47],
[19,20,40,42,47],
[1,8,9,12,37],
[26,31,33,39,40],
[2,9,17,25,29],
[13,18,20,38,39],
[8,26,32,44,50],
[8,15,34,44,49],
[11,17,20,22,29],
[14,24,27,35,39],
[3,15,22,26,37],
[3,7,27,43,46],
[5,13,25,33,35],
[22,25,41,45,49],
[4,18,35,41,43],
[6,16,18,29,35],
[11,25,32,42,47],
[1,8,15,19,37],
[10,11,25,32,49],
[2,21,30,39,43],
[16,29,38,42,48],
[13,19,20,27,41],
[1,4,14,32,48],
[2,13,14,30,32],
[3,4,8,11,41],
[3,19,22,37,39],
[3,7,14,46,48],
[1,16,18,29,44],
[9,28,30,43,49],
[9,16,17,20,32],
[1,10,25,39,44],
[9,11,13,35,38],
[3,26,31,33,50],
[5,34,37,43,48],
[1,18,26,40,47],
[5,6,13,20,33],
[4,17,27,33,46],
[8,24,25,30,50],
[19,31,32,40,46],
[12,14,18,38,46],
[8,23,24,39,50],
[1,6,16,28,38],
[19,38,40,41,50],
[17,23,27,30,39],
[12,14,20,21,39],
[14,27,28,35,47],
[1,31,40,41,45],
[12,35,36,39,47],
[6,21,30,37,44],
[6,12,15,20,30],
[11,19,24,46,49],
[1,9,15,22,49],
[1,7,27,28,41],
[13,20,39,40,42],
[5,8,33,34,46],
[12,23,37,41,46],
[7,22,27,30,31],
[3,7,13,41,48],
[3,5,13,16,30],
[7,9,28,39,47],
[13,14,23,26,31],
[11,19,38,42,46],
[16,18,34,36,49],
[5,15,16,36,37],
[2,4,8,22,24],
[19,23,26,27,49],
[1,18,20,31,36],
[2,7,13,26,49],
[14,16,26,34,38],
[34,36,37,39,49],
[11,19,27,32,42],
[8,9,33,34,36],
[11,14,35,44,45],
[1,2,5,24,47],
[4,23,25,37,39],
[3,19,26,29,36],
[3,20,32,40,42],
[10,25,35,37,43],
[7,20,21,33,41],
[10,12,28,29,49],
[9,11,19,42,45],
[2,5,10,13,28],
[10,16,31,37,40],
[10,20,35,47,50],
[9,10,19,20,35],
[6,12,21,40,49],
[1,2,16,31,50],
[1,5,17,39,46],
[5,28,31,33,49],
[10,14,20,24,40],
[12,24,38,45,46],
[6,7,36,39,40],
[6,19,23,33,34],
[4,10,26,33,48],
[13,20,32,34,47],
[11,12,26,28,37],
[16,18,19,23,44],
[19,21,24,41,48],
[29,37,38,47,50],
[4,19,21,31,42],
[12,22,36,38,50],
[6,14,20,26,46],
[4,8,22,36,44],
[5,11,12,30,40],
[6,15,18,21,38],
[22,33,42,43,46],
[4,8,9,33,43],
[15,28,30,42,50],
[11,27,34,35,39],
[15,28,30,37,43],
[1,7,23,25,45],
[4,28,29,39,44],
[1,10,20,43,44],
[7,10,25,39,42],
[14,16,28,34,35],
[1,4,6,12,13],
[14,15,16,22,50],
[1,16,18,42,47],
[7,10,16,32,40],
[19,28,34,45,49],
[1,20,39,48,50],
[12,17,29,44,50],
[2,3,24,25,34],
[13,27,30,42,46],
[27,30,45,47,50],
[7,14,23,27,35],
[2,21,26,44,45],
[5,23,25,35,40],
[12,15,19,29,48],
[4,5,17,39,47],
[13,23,31,42,44],
[7,18,19,40,49],
[1,7,13,19,20],
[3,7,16,18,25],
[13,18,20,35,46],
[3,16,25,38,45],
[25,26,30,36,44],
[8,14,34,40,44],
[10,27,43,45,46],
[6,22,33,46,49],
[27,31,44,46,49],
[3,38,39,41,45],
[8,11,23,41,42],
[7,14,31,35,46],
[22,33,41,46,50],
[3,11,13,15,23],
[2,4,17,20,46],
[13,17,20,24,47],
[1,24,25,31,38],
[6,14,19,27,35],
[1,3,11,18,31],
[15,16,17,23,30],
[3,9,25,31,49],
[27,35,36,38,48],
[14,18,26,40,45],
[18,39,44,46,47],
[13,14,21,23,40],
[1,4,20,32,34],
[11,20,28,41,45],
[3,4,9,19,28],
[11,38,40,42,48],
[15,22,29,33,47],
[10,17,32,36,47],
[2,12,15,29,44],
[5,17,28,40,44],
[9,40,43,44,46],
[1,10,25,46,49],
[2,20,23,29,50],
[2,35,44,45,50],
[1,14,17,24,50],
[3,4,17,41,47],
[11,16,18,22,43],
[5,16,20,29,30],
[9,15,20,24,34],
[1,7,12,16,18],
[16,26,32,40,47],
[16,30,33,40,43],
[2,7,38,40,45],
[16,17,25,40,44],
[3,9,17,45,47],
[10,23,26,29,35],
[15,24,29,33,41],
[7,8,24,34,46],
[4,8,19,25,44],
[18,26,33,42,46],
[16,18,20,27,46],
[15,23,28,33,36],
[4,27,37,48,49],
[4,14,22,33,42],
[5,15,17,29,32],
[6,8,16,23,50],
[3,10,21,25,34],
[22,24,25,28,46],
[9,21,31,32,33],
[24,26,29,36,49],
[3,14,17,37,39],
[1,11,23,41,44],
[15,31,35,40,46],
[14,19,21,30,32],
[5,13,22,36,39],
[1,5,7,9,21],
[16,24,33,35,43],
[9,18,30,47,48],
[2,7,24,38,45],
[13,14,21,34,46],
[13,33,40,42,43],
[2,22,40,43,50],
[4,8,12,25,31],
[4,15,17,21,23],
[2,12,32,43,44],
[8,25,26,38,48],
[3,6,9,18,24],
[4,7,28,36,43],
[24,33,35,46,49],
[5,8,16,42,46],
[3,8,13,18,40],
[12,15,32,44,49],
[6,26,31,42,50],
[9,12,28,32,48],
[18,19,33,38,44],
[5,17,27,33,42],
[8,32,34,46,49],
[13,15,18,39,45],
[17,22,28,31,46],
[18,22,35,36,41],
[4,16,21,31,42],
[4,14,24,26,30],
[3,16,20,34,49],
[19,24,31,38,40],
[3,10,25,32,43],
[6,12,35,39,49],
[2,9,23,36,47],
[24,25,28,35,48],
[6,29,38,45,47],
[5,8,21,24,26],
[1,24,30,31,47],
[14,16,21,25,26],
[7,16,18,19,24],
[4,29,30,31,45],
[1,2,11,19,47],
[15,20,24,44,49],
[4,9,15,24,42],
[22,31,43,44,50],
[14,16,21,30,37],
[18,21,37,43,47],
[1,6,11,17,38],
[3,9,10,19,42],
[5,7,15,19,29],
[20,27,33,35,46],
[8,26,38,47,50],
[10,12,35,36,43],
[7,8,20,35,38],
[8,23,40,41,42],
[20,27,37,41,45],
[18,25,26,35,38],
[1,15,34,48,50],
[31,36,40,42,45],
[31,32,45,47,49],
[3,8,30,46,48],
[6,11,38,41,44],
[2,4,20,21,49],
[7,20,35,42,44],
[15,18,19,41,42],
[10,18,32,35,46],
[21,24,29,30,50],
[3,21,22,33,47],
[17,21,41,48,49],
[21,24,26,34,47],
[15,19,20,45,49],
[8,12,13,39,44],
[6,9,31,43,44],
[2,30,34,35,45],
[3,17,31,34,40],
[14,20,23,39,49],
[10,19,24,30,39],
[3,12,24,37,38],
[2,3,30,31,45],
[8,14,23,30,45],
[25,31,38,49,50],
[12,20,21,22,35],
[7,12,28,34,45],
[6,27,30,35,41],
[4,14,25,34,49],
[1,23,32,45,49],
[5,12,20,29,48],
[1,7,12,23,39],
[7,16,22,36,44],
[2,6,30,32,49],
[2,13,39,45,47],
[12,22,24,29,38],
[15,19,35,36,41],
[1,17,29,39,42],
[9,14,28,30,37],
[13,19,23,34,41],
[3,21,26,40,41],
[2,7,8,43,50],
[1,18,23,33,41],
[6,13,15,34,35],
[6,11,12,21,41],
[9,11,15,36,43],
[21,27,29,34,49],
[12,15,32,40,45],
[8,22,31,32,36],
[9,16,17,29,39],
[7,16,22,30,48],
[2,22,33,38,47],
[14,16,32,34,47],
[12,34,36,47,48],
[5,6,9,15,29],
[13,21,25,34,35],
[7,11,19,32,43],
[3,19,28,43,49],
[4,9,15,24,28],
[9,20,27,35,48],
[26,27,30,46,49],
[8,11,22,38,41],
[5,23,28,38,49],
[2,5,24,43,45],
[7,12,14,40,42],
[1,7,9,29,46],
[15,19,34,39,49],
[1,11,17,23,29],
[5,11,35,44,50],
[17,21,23,37,45],
[11,19,24,33,39],
[5,12,26,47,50],
[5,17,21,37,38],
[18,20,34,49,50],
[1,2,22,25,30],
[1,27,37,40,41],
[4,13,15,41,49],
[4,17,27,28,50],
[7,10,19,26,42],
[17,36,38,43,46],
[8,22,25,38,50],
[10,19,32,36,46],
[38,40,41,46,48],
[16,30,33,36,43],
[2,3,16,33,46],
[15,18,24,27,44],
[27,35,36,38,41],
[3,13,24,29,32],
[9,23,34,40,42],
[6,11,18,26,34],
[3,11,19,34,37],
[7,38,41,44,50],
[1,3,4,36,43],
[11,18,23,29,32],
[12,19,20,28,31],
[3,6,11,14,49],
[23,27,34,40,43],
[3,7,19,27,29],
[1,15,29,42,50],
[2,12,15,33,39],
[15,26,35,37,43],
[13,17,26,49,50],
[8,20,23,48,50],
[11,14,44,46,49],
[4,17,22,30,47],
[4,5,10,25,31],
[8,14,15,20,31],
[9,14,24,37,39],
[13,33,42,48,50],
[5,14,39,43,44],
[16,19,21,29,36],
[4,31,39,43,46],
[8,31,34,36,45],
[19,20,22,25,42],
[5,9,20,44,48],
[2,14,18,23,42],
[14,17,20,27,32],
[12,22,35,38,49],
[20,33,34,37,39],
[20,28,32,38,46],
[2,6,8,21,25],
[15,33,34,38,43],
[11,23,37,38,44],
[6,13,25,31,49],
[4,30,43,44,46],
[6,12,20,21,34],
[7,17,21,37,39],
[2,5,13,15,23],
[8,9,30,34,48],
[6,8,16,44,50],
[3,17,21,35,42],
[7,16,36,42,43],
[8,17,21,23,47],
[9,15,27,41,44],
[2,5,9,29,32],
[5,21,23,29,35],
[12,18,27,33,41],
[5,10,26,37,42],
[5,10,25,29,32],
[4,19,34,41,43],
[1,17,20,36,49],
[5,31,39,46,49],
[1,8,33,38,43],
[11,20,31,35,46],
[10,15,18,24,39],
[7,8,35,37,49],
[9,10,28,38,48],
[2,26,41,45,48],
[19,20,24,36,41],
[2,5,10,20,24],
[3,7,15,17,35],
[2,18,28,42,50],
[10,27,32,41,49],
[5,6,17,18,34],
[6,13,22,24,30],
[5,6,39,49,50],
[3,7,34,43,50],
[7,13,30,43,47],
[2,13,41,45,50],
[4,22,28,32,47],
[25,26,40,45,47],
[8,26,29,41,48],
[14,30,37,39,50],
[4,22,27,39,41],
[5,7,45,48,49],
[3,21,23,28,46],
[11,28,30,35,50],
[1,2,23,43,45],
[6,10,17,29,49],
[13,24,30,35,48],
[2,10,23,29,50],
[4,10,24,34,35],
[7,16,28,36,43],
[6,10,22,30,36],
[3,16,26,30,47],
[9,11,16,19,32],
[20,23,24,37,43],
[1,11,17,19,33],
[1,2,7,24,25],
[14,26,29,46,50],
[6,11,39,40,47],
[10,12,17,31,49],
[23,26,36,40,44],
[14,32,34,38,46],
[4,17,32,34,49],
[4,13,32,39,41],
[2,9,18,21,39],
[2,6,18,29,37],
[18,27,41,45,49],
[2,5,11,22,24],
[6,10,11,20,38],
[9,18,26,41,43],
[20,22,26,34,40],
[16,25,27,41,45],
[2,13,22,36,40],
[20,30,38,39,44],
[15,23,26,31,44],
[8,12,15,17,46],
[17,42,43,48,49],
[16,17,26,30,35],
[7,35,36,39,47],
[3,5,8,10,44],
[13,18,24,34,50],
[2,11,18,47,49],
[17,26,35,37,39],
[4,5,21,30,43],
[18,32,39,42,44],
[7,9,40,48,49],
[15,17,23,35,38],
[11,15,24,28,41],
[1,7,11,33,48],
[13,14,25,28,42],
[11,20,23,37,46],
[4,6,8,42,48],
[9,16,32,37,46],
[6,12,25,48,49],
[8,12,37,44,47],
[6,8,13,21,32],
[3,13,33,36,47],
[9,14,15,20,47],
[6,11,26,43,49],
[9,34,35,42,44],
[28,33,34,37,44],
[3,37,45,47,50],
[28,29,31,37,50],
[14,28,31,47,50],
[9,16,27,41,45],
[7,14,34,41,49],
[6,11,16,35,44],
[1,12,15,31,47],
[9,16,17,27,31],
[4,9,29,34,37],
[20,21,30,41,43],
[1,7,17,44,50],
[10,15,25,37,46],
[3,10,20,36,42],
[1,2,6,14,45],
[5,12,15,21,39],
[13,20,34,38,43],
[1,5,12,18,20],
[6,20,27,38,49],
[1,18,37,46,48],
[4,11,12,16,42],
[6,17,22,39,46],
[9,16,32,34,48],
[5,18,21,29,45],
[8,13,16,44,47],
[6,21,23,26,43],
[6,12,36,37,44],
[10,11,31,37,44],
[1,15,19,24,33],
[16,28,32,36,48],
[5,19,33,37,42],
[7,11,20,21,29],
[16,28,31,35,42],
[5,13,16,41,45],
[12,21,24,28,40],
[4,8,9,30,35],
[11,12,13,23,26],
[17,18,30,33,35],
[6,11,29,34,39],
[10,27,30,32,34],
[28,30,31,45,46],
[1,5,8,20,35],
[1,2,11,14,36],
[1,3,29,45,47],
[2,8,16,21,39],
[8,9,11,13,50],
[5,7,21,22,29],
[8,13,24,35,46],
[11,29,32,46,47],
[5,19,33,36,42],
[9,18,30,34,48],
[2,3,18,23,39],
[7,10,13,34,47],
[3,14,23,41,43],
[19,21,23,36,39],
[9,18,20,40,41],
[11,15,17,24,46],
[2,4,12,31,50],
[14,16,24,40,43],
[7,16,22,38,41],
[3,9,11,20,39],
[10,16,34,36,49],
[2,9,38,40,44],
[3,17,19,32,38],
[6,12,25,31,37],
[1,35,36,38,39],
[12,21,23,26,41],
[2,16,22,28,46],
[8,40,41,46,47],
[2,5,11,27,38],
[14,18,20,39,42],
[1,13,16,23,27],
[9,11,13,15,25],
[21,29,31,46,49],
[5,13,43,45,50],
[3,8,10,31,36],
[8,18,26,38,39],
[14,24,31,44,45],
[23,24,38,42,44],
[14,24,29,45,48],
[13,28,29,31,47],
[17,18,40,43,50],
[7,8,12,21,43],
[16,23,30,37,41],
[6,8,42,49,50],
[11,16,22,34,46],
[5,14,35,40,47],
[6,13,15,20,40],
[6,15,21,34,48],
[6,21,23,31,39],
[26,36,43,47,49],
[1,17,22,29,31],
[9,30,34,38,48],
[4,14,15,20,28],
[9,20,21,22,38],
[16,27,33,34,39],
[11,12,13,23,36],
[23,32,38,45,49],
[6,11,16,26,49],
[2,8,28,32,37],
[2,8,20,34,40],
[7,15,17,18,39],
[4,6,17,31,45],
[4,6,12,31,38],
[4,13,16,22,27],
[2,21,34,40,48],
[19,26,36,48,49],
[11,30,32,45,47],
[9,12,26,41,47],
[3,31,34,43,45],
[6,19,32,39,42],
[10,12,18,33,47],
[9,18,20,32,39],
[18,23,35,37,41],
[10,12,15,46,48],
[13,17,21,30,39],
[16,19,20,26,44],
[4,10,11,20,22],
[7,20,22,45,48],
[7,11,17,18,34],
[1,3,11,15,30],
[10,19,22,37,41],
[15,17,30,38,49],
[13,26,30,34,41],
[2,20,30,31,40],
[2,11,17,23,49],
[2,8,11,16,20],
[16,20,25,30,49],
[1,20,28,32,49],
[5,17,36,37,50],
[12,15,17,30,32],
[7,11,30,31,39],
[14,17,29,32,45],
[5,8,16,30,37],
[1,7,21,27,43],
[1,34,39,47,49],
[35,36,37,41,48],
[8,14,21,34,36],
[2,3,6,15,35],
[3,18,23,29,47],
[4,20,33,37,45],
[9,17,36,40,45],
[3,11,32,33,35],
[28,31,39,45,49],
[19,22,23,24,27],
[1,2,29,36,48],
[7,23,31,33,38],
[2,3,4,21,45],
[13,26,27,35,46],
[4,23,34,39,45],
[1,3,24,43,49],
[8,15,29,37,45],
[4,12,16,29,31],
[10,21,27,42,46],
[4,10,23,24,45],
[2,22,24,30,40],
[8,14,25,31,45],
[1,8,30,43,45],
[10,29,30,32,40],
[4,11,16,25,32],
[5,14,25,26,44],
[2,14,30,32,34],
[8,22,27,36,43],
[13,18,22,26,32],
[9,13,21,24,38],
[7,11,22,26,46],
[2,16,30,31,49],
[14,20,26,30,31],
[4,9,22,32,35],
[15,18,25,29,35],
[6,9,33,34,50],
[4,11,16,46,50],
[25,28,29,31,33],
[13,21,22,26,48],
[15,24,29,33,39],
[8,11,25,31,48],
[5,17,23,36,37],
[7,11,27,42,45],
[7,10,31,41,46],
[2,3,17,40,44],
[1,3,13,24,44],
[9,17,19,26,39],
[3,13,34,41,43],
[6,15,25,29,41],
[17,37,42,45,50],
[4,16,27,34,44],
[13,29,42,44,48],
[8,11,23,44,45],
[9,20,38,44,45],
[2,4,23,30,40],
[4,32,36,38,47],
[2,19,36,42,50],
[3,17,26,30,49],
[13,21,27,28,41],
[22,29,36,38,43],
[6,23,38,42,45],
[2,3,34,38,49],
[27,31,35,46,50],
[4,7,19,26,27],
[6,10,30,34,41],
[20,21,28,32,37],
[10,19,24,25,40],
[7,20,23,24,37],
[8,14,45,47,50],
[17,23,30,41,43],
[1,4,19,35,42],
[11,14,18,35,42],
[1,3,10,32,44],
[9,15,28,36,39],
[9,15,28,36,39],
[1,9,25,27,37],
[2,21,26,34,49],
[1,20,21,27,29],
[1,16,20,23,44],
[17,34,38,42,48],
[10,11,17,20,30],
[7,9,14,18,31],
[3,17,22,28,40],
[2,9,16,46,47]
]
data = pd.DataFrame(data_content, columns=range(5))

# Skalowanie danych
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data)

# Parametry modelu
input_dim = X_scaled.shape[1]
latent_dim = 12
num_mixtures = 5

# Funkcja straty MDN
def mdn_loss(num_mixtures, output_dim):
    def loss(y_true, outputs):
        alphas, mus, sigmas = tf.split(outputs, [
            num_mixtures, num_mixtures * output_dim, num_mixtures * output_dim
        ], axis=-1)
        mus = tf.reshape(mus, [-1, num_mixtures, output_dim])
        sigmas = tf.reshape(sigmas, [-1, num_mixtures, output_dim])
        y_true = tf.expand_dims(y_true, axis=1)

        gaussians = tf.exp(
            -0.5 * tf.reduce_sum(tf.square(
                (y_true - mus) / sigmas), axis=-1)) / (
                    tf.reduce_prod(sigmas, axis=-1) * tf.sqrt(2.0 * np.pi))
        weighted_gaussians = alphas * gaussians
        nll = -tf.math.log(tf.reduce_sum(weighted_gaussians, axis=-1) + 1e-8)
        return tf.reduce_mean(nll)

    return loss

# Tworzenie modelu MDN + VAE
def create_mdn_vae_model(input_dim, latent_dim, num_mixtures):
    inputs = Input(shape=(input_dim, ))
    h = Dense(64, activation='relu')(inputs)
    h = Dense(32, activation='relu')(h)
    z_mean = Dense(latent_dim, name='z_mean')(h)
    z_log_var = Dense(latent_dim, name='z_log_var')(h)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(latent_dim, ),
               name='z')([z_mean, z_log_var])

    h_decoder = Dense(32, activation='relu')(z)
    h_decoder = Dense(64, activation='relu')(h_decoder)

    alphas = Dense(num_mixtures, activation='softmax',
                   name='alphas')(h_decoder)
    mus = Dense(num_mixtures * input_dim, name='mus')(h_decoder)
    sigmas = Dense(num_mixtures * input_dim,
                   activation='softplus',
                   name='sigmas')(h_decoder)

    outputs = Lambda(lambda x: tf.concat(x, axis=-1),
                     name='mdn_output')([alphas, mus, sigmas])

    mdn_vae = Model(inputs, outputs, name='mdn_vae')
    return mdn_vae

# Kompilacja modelu
model = create_mdn_vae_model(input_dim, latent_dim, num_mixtures)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=mdn_loss(num_mixtures, input_dim))

# {ON  DISPLAY} Wyłączenie wyświetlania w konsoli
sys.stdout = open(os.devnull, 'w')

# Trening
model.fit(X_scaled, X_scaled, epochs=iRow, batch_size=100, validation_split=0.2, verbose=1)

# {ON  DISPLAY} Włączenie wyświetlania wyników w konsoli
sys.stdout = sys.__stdout__

# import sys
# sys.exit()

# Funkcja oceny próbki
def evaluate_sample(model, scaler, sample, target_pattern):
    output = model.predict(sample, verbose=0)
    alphas, mus, sigmas = tf.split(output, [
        num_mixtures, num_mixtures * input_dim,
        num_mixtures * input_dim
    ], axis=-1)

    mus = tf.reshape(mus, [-1, num_mixtures, input_dim])
    prediction = scaler.inverse_transform(tf.reshape(mus[0, 0, :], (1, -1)).numpy())
    generated_numbers = sorted(np.round(np.clip(prediction[0], 1, 50)).astype(int))

    # Dopasowanie liczby elementów w generated_numbers do target_pattern
    if len(generated_numbers) > len(target_pattern):
        generated_numbers = generated_numbers[:len(target_pattern)]  # Przycięcie
    elif len(generated_numbers) < len(target_pattern):
        generated_numbers += [0] * (len(target_pattern) - len(generated_numbers))  # Uzupełnienie zerami

    score = np.sum(np.abs(np.array(generated_numbers) - np.array(target_pattern)))
    return -score

# Algorytm Metropolisa-Hastingsa
def metropolis_hastings(model, scaler, target_pattern, num_samples=iRow, input_dim=5):
    current_sample = np.random.normal(size=(1, input_dim))
    accepted_samples = []
    current_score = evaluate_sample(model, scaler, current_sample, target_pattern)

    with tqdm(total=num_samples, desc="MCMC Progress", unit="sample") as pbar:
        for _ in range(num_samples):
            proposed_sample = current_sample + np.random.normal(0, 0.1, size=current_sample.shape)
            proposed_score = evaluate_sample(model, scaler, proposed_sample, target_pattern)
            acceptance_prob = min(1, np.exp(current_score - proposed_score))

            if np.random.rand() < acceptance_prob:
                current_sample = proposed_sample
                current_score = proposed_score
                accepted_samples.append(current_sample)

            pbar.update(1)

    return np.array(accepted_samples)

# Szukanie wzorca za pomocą MCMC
def find_pattern_with_mcmc(model, scaler, target_pattern, max_samples=iRow):
    print(f"Rozpoczynam wyszukiwanie wzorca {target_pattern} przy użyciu MCMC...")

    accepted_samples = metropolis_hastings(model, scaler, target_pattern, num_samples=max_samples)

    for sample in accepted_samples:
        generated_numbers = sorted(np.round(np.clip(sample[0], 1, 50)).astype(int))
        if generated_numbers == target_pattern:
            print(f"\nZnaleziono wzorzec: {generated_numbers}")
            break
    else:
        print("\nNie znaleziono wzorca w wygenerowanych próbkach.")

print(f"Szukanie wzorca dla seeda1: {seed1_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
find_pattern_with_mcmc(model, scaler, target_pattern)

print(f"\nSzukanie wzorca dla seeda2: {seed2_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
find_pattern_with_mcmc(model, scaler, target_pattern)

# Hello Qubit

# Nowa sekcja

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/start/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

In [ ]:
# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit that applies a square root of NOT gate, then measures the qubit.
circuit = cirq.Circuit(cirq.X(qubit) ** 0.5, cirq.measure(qubit, key='m'))
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

# Congratulations
You've just run your first Cirq program.

To learn about running a circuit on a virtual machine that mimics existing quantum hardware, see [Quantum Virtual Machine](../simulate/quantum_virtual_machine.ipynb).

If you would like to learn more about quantum computing, check out our [education page](https://quantumai.google/resources). The Full API reference for Cirq can be found [here](/reference/python/cirq). If you are looking for vendor specific information that can be found on our vendor sub-pages:


  [Alpine Quantum Technologies](../hardware/aqt/getting_started.ipynb)
  
  [Pasqal](../hardware/pasqal/getting_started.ipynb)
  
  [IonQ](../hardware/ionq/getting_started.ipynb)
  
  [Azure](../hardware/azure-quantum/getting_started_honeywell.ipynb)
  
  [Rigetti](../hardware/rigetti/getting_started.ipynb)